### Download data

- Lower file size from 17GB to 1.5GB
    - Remove some unnecessary data
    - Compress as xz 

In [1]:
# !wget https://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gaf.gz  -O goa_uniprot_all_ebi.gaf.gz.download
# !mv goa_uniprot_all_ebi.gaf.gz.download goa_uniprot_all_ebi.gaf.gz
# !gunzip -c goa_uniprot_all_ebi.gaf.gz | awk 'BEGIN {OFS="\t";FS="\t"} ($1 == "UniProtKB") {print $2,$4,$5,$7,$9,$14}' | sort -u | xz -T0 > goa_uniprot_all_ebi_filtered.tsv.xz.tmp
# !mv goa_uniprot_all_ebi_filtered.tsv.xz.tmp ../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz

In [2]:
# TODO oneliner, did it work?
# wget https://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gaf.gz | gunzip -c | awk 'BEGIN {OFS="\t";FS="\t"} ($1 == "UniProtKB") {print $2,$4,$5,$7,$9,$14}' | sort -u | xz -T0 > goa_uniprot_all_ebi_filtered.tsv.xz.tmp && mv goa_uniprot_all_ebi_filtered.tsv.xz.tmp goa_uniprot_all_ebi_filtered.tsv.xz

### Read into pandas

Using optimized dtypes to save space

In [3]:
import pandas as pd
from subpred.util import save_df
DATASET_FOLDER = "../data/datasets"

In [4]:
df_go_ebi = pd.read_table(
    "../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz",
    # force_update=True,
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
    dtype={
        "Uniprot": "string",
        "qualifier": "category",
        "go_id": "string",
        "evidence_code": "category",
        "aspect": "category",
        "date": "int",
    },
    # parse_dates=["date"],
)


In [ ]:
# TODO category?
df_go_ebi.date.value_counts()

2023-01-25    684773931
2023-01-27     20129489
2017-02-28       635905
2022-10-26       371126
2023-01-11       239317
                ...    
2019-09-08            1
2005-12-18            1
2022-06-25            1
2015-12-12            1
2008-02-03            1
Name: date, Length: 7197, dtype: int64

In [ ]:
print(df_go_ebi.shape)
df_go_ebi = df_go_ebi.drop("date", axis=1)
df_go_ebi = df_go_ebi.drop_duplicates()
print(df_go_ebi.shape)

(710667178, 6)
(710498697, 5)


### Save pickles containing datasets:


In [ ]:
save_df(df_go_ebi, "go_all", DATASET_FOLDER)

Filtered dataframe with no IEA terms

In [ ]:
df_go_ebi_mf_enables = df_go_ebi[
    (df_go_ebi.qualifier == "enables")&
    (df_go_ebi.aspect == "F")
].drop(["qualifier", "aspect"], axis=1)
save_df(df_go_ebi_mf_enables, "go_mf", DATASET_FOLDER)

In [ ]:
!du -hs ../data/datasets/*

9.9G	../data/datasets/go_all.pickle
7.9G	../data/datasets/go_mf.pickle
656K	../data/datasets/interpro.pickle
40K	../data/datasets/keywords.pickle
158M	../data/datasets/uniprot70.pickle
498M	../data/datasets/uniprot.pickle


In [ ]:
# df_go_ebi_filtered = df_go_ebi[df_go_ebi.Uniprot.isin(df_uniprot.index)]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.qualifier == "enables"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.evidence_code != "IEA"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.aspect == "F"]
# df_go_ebi_filtered = df_go_ebi_filtered.drop(["qualifier", "aspect", "date"], axis=1).reset_index(drop=True)